<a href="https://colab.research.google.com/github/JunhyeongPark-kr/YAI-Basic-Study-Toy-Project/blob/main/STS_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
!cd /content/drive/MyDrive/YAI

In [67]:
! pip install sentence_transformers datasets

In [68]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, models, LoggingHandler, losses, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [69]:
# logger
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [70]:
pretrained_model_name = 'klue/roberta-base'
sts_num_epochs = 10
train_batch_size = 32

sts_model_save_path = 'output/training_sts-'+pretrained_model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# 1. Load Dataset & Preprocessing

## 1.1. KLUE-STS

In [71]:
# load KLUE-STS Dataset
klue_sts_train = load_dataset("klue", "sts", split='train[:90%]')
klue_sts_valid = load_dataset("klue", "sts", split='train[-10%:]') # train의 10%를 validation set으로 사용
klue_sts_test = load_dataset("klue", "sts", split='validation')

print('Length of Train : ',len(klue_sts_train))
print('Length of Valid : ',len(klue_sts_valid))
print('Length of Test : ',len(klue_sts_test))

Length of Train :  10501
Length of Valid :  1167
Length of Test :  519


In [72]:
klue_sts_train[0]

{'guid': 'klue-sts-v1_train_00000',
 'source': 'airbnb-rtt',
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}}

In [73]:
def make_sts_input_example(dataset):
    '''
    Transform to InputExample
    '''
    input_examples = []
    for i, data in enumerate(dataset):
        sentence1 = data['sentence1']
        sentence2 = data['sentence2']
        score = (data['labels']['label']) / 5.0  # normalize 0 to 5
        input_examples.append(InputExample(texts=[sentence1, sentence2], label=score))

    return input_examples

In [74]:
sts_train_examples = make_sts_input_example(klue_sts_train)
sts_valid_examples = make_sts_input_example(klue_sts_valid)
sts_test_examples = make_sts_input_example(klue_sts_test)

In [75]:
# Train Dataloader
train_dataloader = DataLoader(
    sts_train_examples,
    shuffle=True,
    batch_size=train_batch_size,
)

# Evaluator by sts-validation
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    sts_valid_examples,
    name="sts-dev",
)

# Evaluator by sts-test
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    sts_test_examples,
    name="sts-test",
)

# 2. Load Embedding Model

In [76]:
# Load Embedding Model
embedding_model = models.Transformer(
    model_name_or_path=pretrained_model_name,
    max_seq_length=256,
    do_lower_case=True
)

# Only use Mean Pooling -> Pooling all token embedding vectors of sentence.
pooling_model = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

model = SentenceTransformer(modules=[embedding_model, pooling_model])

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 3. STS training

In [77]:
# Use CosineSimilarityLoss
train_loss = losses.CosineSimilarityLoss(model=model)

# warmup steps
warmup_steps = math.ceil(len(sts_train_examples) * sts_num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Training
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=dev_evaluator,
    epochs=sts_num_epochs,
    evaluation_steps=int(len(train_dataloader)*0.1),
    warmup_steps=warmup_steps,
    output_path=sts_model_save_path
)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/329 [00:00<?, ?it/s]

Iteration:   0%|          | 0/329 [00:00<?, ?it/s]

Iteration:   0%|          | 0/329 [00:00<?, ?it/s]

Iteration:   0%|          | 0/329 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/329 [00:00<?, ?it/s]

Iteration:   0%|          | 0/329 [00:00<?, ?it/s]

Iteration:   0%|          | 0/329 [00:00<?, ?it/s]

Iteration:   0%|          | 0/329 [00:00<?, ?it/s]

Iteration:   0%|          | 0/329 [00:00<?, ?it/s]

# 4. Evaluation

In [78]:
# evaluation sts-test
test_evaluator(model, output_path=sts_model_save_path)

0.8921760072694922

In [79]:
model.save('/content/drive/MyDrive/YAI/STSmodel.h5')

In [80]:
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [81]:
# 1. 모델 로드
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 2. 비교할 문장
sentence1 = "이게 맞냐?"
sentence2 = "너 이게 맞다고 생각해?"

# 3. 문장 벡터화
embedding1 = model.encode(sentence1)
embedding2 = model.encode(sentence2)

# 4. 유사도 계산 (코사인 유사도)
cosine_similarity = util.pytorch_cos_sim(embedding1, embedding2)

print("문장 간 유사도:", cosine_similarity.item())

문장 간 유사도: 0.9097855687141418


In [82]:
!pip install faiss-gpu

# Dataset

In [83]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import faiss
import re

papers_dataset = {
    "doc_id": "JAKO200531559976988",
    "title": {
      "ko": "엘자 스키아빠렐리의 의상(衣裳)에 나타난 장식요소(裝飾要素)의 상징체계(象徵體系)",
      "en": "A Study in the Symbol System of Clothing Decorations in Elsa Schiaparelli's Design Works"
    },
    "authors": "백정현;배수정",
    "journal": {
      "ko": "패션비즈니스",
      "en": "Journal of Fashion Business"
    },
    "year": "2005",
    "abstract": {
      "en": "The aim of this study is to find out the symbol of clothing decorations in Elsa Schiaparalli's design works and there are four major points due to the aim of this study. Firstly, a fantasy is represented through Surrealistic Arts which creates mysterious, secrete, and surprising spirits. In Surrealistic Arts, the fashion of schiaparelli demonstrates a fantasy spirit by using the methods like metaphor, transformation, and re-positioning. Secondly, In Surrealistic paintings, normally double image or different image from symbol immanent were expressed. However, Elsa Schiaparelli used double and multi-image decorations instead that has well-organized formative effect. The mixture of double images can be separated as symbolic mixture, design mixture and expressive mixture. Thirdly, the body parts has represented symbolism and sensuality in Surrealism Arts. Elsa Schiaparelli has demonstrated the expression of modern clothing as passionate, desirable, and powerful. This is the reason why her designs were absolutely different from the previous sihouette-focused clothing. Fourth, there are lots of intentional decoration that are different from actual images, such as transformation, exaggeration, minimization and repettion, as well as, re-location, re-arrangement, line-up arrangement, collage and odd materials."
    },
    "keywords": {
      "ko": "초현실주의;환상성;이중성;관능성;유희성;",
      "en": "surrealism;fantasy;dualism;dualism;amusement;"
    },
    "body_text": [
      {
        "section": "Ⅰ. 서론"
      },
      {
        "section": "1. 연구의 목적",
        "text": [
          "현대의상은 인간의 내적 미의식을 탐구하는 철학적 접근과 더불어 발전하는 소재와 기술, 다양한 창의적 표현을 통해 조형적 아름다움을 추구하고 있으며, 다른 예술 장르와 결합함으로써 그 범위를 점차 넓혀가고 있다.",
          "엘자 스키아빠렐리(Elsa Schiaparelli, 1890~1973)는 대담한 독창성과 혁신적인 창조력, 초현실주의 철학에 대한 이해로 아티스트 꾸띠뤼에(artist-couturier)의 전형이 되었다1). 그녀는 살바도르 달리(Salvador Dali, 1904~1989), 장 꼭도(Jean Cocteau, 1886~1963), 포토그래퍼 세실 비통(Cecil Beaton, 1904~1980)과 같은 예술가들과 교류하면서 초현실주의적 의미와 기법을 의상에 표현한 디자인들로 1930년대 패션의 변화를 주도하였고2) 유머로서의 패션을 제시하였으며 현대의상을 시대의 예술과 그 정신을 표현하는 창의적 산물이라는 새로운 개념으로 확장시키는데 지대한 업적을 남겼다. 더불어 예술과의 조우를 통해 익혀진 조형감각과 숙련된 기술, 다양한 사물과 소재의 활용을 통해 장식적 측면의 중요성을 인식시키고 그 깊이를 더하는데 이바지하였다.",
          "스키아빠렐리에 대한 선행연구들은 '현대패션에 나타난 트롱쁘-뢰이유(Trompe-loeil, 눈속임)에 관한 연구3)’, ‘Schiaparelli의 의상에 나타난 초현실적 표현에 관한 연구4)’에서 보여진 바와 같이 주로 초현실주의 회화 기법과 관련된 작품의 특성에 중점을 둔 방향으로 진행되어져 왔다. 이러한 선행연구를 바탕으로 본 연구는 스키아빠렐리의 의상이 단순하고 현대적인 감각을 지닌 디자인의 의상에 회화적 기법이나 입체적 장식물 또는 악세서리들을 통한 장식적 표현으로 독특한 미감을 형성한 배경과 그 속에 내재된 상징체계를 밝히고자 하였다. 따라서 본 연구의 목적은 엘자 스키아빠렐리의 의복 디자인에서 볼 수 있는 장식요소의 특성들을 그 표현형식에 따라 분류하고 그 상징체계와 의미를 분석하는데 있다. 이러한 시도는 현대의상이 다른 예술 장르와 융합하여 새로운 창의적 디자인을 고안하는 데 참고 자료가 될 것이라 생각된다."
        ]
      },
      {
        "section": "2. 연구의 내용 및 방법",
        "text": [
          "본 연구는 엘자 스키아빠렐리 의상의 장식요소들을 분석함에 있어 그 시간적 범위를 첫번째 컬렉션이 열렸던 1927년부터 마지막 컬렉션이 열렸던 1949년까지로 하였으며, 구체적인 연구 내용을 살펴보면 2장에서는 엘자 스키아빠렐리 디자인의 배경이 되었던 개인적인 경험에 대해 살펴보았고, 3장에서는 의상의 장식적 요소의 표현형식을 회화적 장식, 입체적 장식, 악세서리를 통한 장식으로 나누어 분석하였다. 그리고 4장에서는 스키아빠렐리의 의상에 나타난 장식요소의 상징체계를 구체적으로 분류하여 살펴보았다 본 논문의 연구방법은 문헌연구방법으로 선행논문과 관련문헌을 중심으로 살펴보았으며 연구자료로는 관련 논문과 문헌 및 패션전문잡지, 관련 인터넷 사이트를 참고하였다."
        ]
      },
      {
        "section": "Ⅱ. 엘자 스키아빠렐리 디자인의 배경",
        "text": [
          "1890년 스키아빠렐리는 로마의 베드로 성당 근교의 르네상스시대 건출물인 Palazzo Corsini 궁에서 태어났다. 그녀의 아버지는 로마의 귀족 출신으로 궁 안의 도서관을 관장하였기 때문에 스키아빠렐리는 궁 안의 미켈란젤로가 디자인한 대리석, 황금과 청동으로 장식된 내부, 프레스코 천정벽화, 장엄한 주홍 보라 조끼를 입은 추기경들과 같이 르네상스 미술을 직접 접하며 성장하였으며 아랍 시인들의 작품, 페르시아 작품연구집과 같은 희귀한 책들을 통해 동양의 신비를 비롯한 다채로운 문화를 접할 수 있었다5). (Camille 그녀는 피사로 Pissarro, 1830~1903)의 태양과 행성을 숭배하는 잉카제국에 감동을 받았고 그들의 보석과 도자기, 직물염색에 매혹되었으며 생동감 넘치는 페루의 분홍색에서 영감을 받았는데 이는 후에 그 유명한 쇼킹 핑크(Shocking Pink)의 고안으로 이어지게 된다6). 또한 스키아빠렐리는 소녀 시절 천체망원경으로 하늘의 별들을 보곤 했으며 천문학자였던 삼촌은 그녀가 뺨에 난 사마귀를 슬퍼했을 때 그것은 아름다움의 표시이며 큰곰자리나 북두칠성을 닮아 그녀가 행운아인 상징이라고 말해 천체와 관련한 상상의 세계를 만들 수 있게 해주었다7). 또한 그녀는 호기심이 왕성하여 소녀 시절 자신을 아름답게 만들기 위해 데이지나 나팔꽃의 씨를 목, 귀, 코, 입 등에 넣어 꽃이 피워지기를 기다리기도 하였는데 이러한 경험은 후에 꽃씨 묶음으로 구성된 ‘Seed Catalogue’ 프린트로 그녀의 작품에 나타나기도 하였다8).",
          "이렇듯 열정적이며 신비적인 성향을 지녔던 그녀는 1914년 런던에서 견신론9)을 강의하는 젊은 신학자에게 매혹되어 결혼 후 뉴욕으로 이주하게 되었으나 방탕한 생활을 일삼는 남편과 그로 인한 생활고, 소아마비에 걸린 딸, 아버지의 죽음 등이 이어지면서 불행했던 결혼생활을 실패로 끝맺게 된다. 자서전을 통해 \"미국은 나에게 독특한 세계를 갖도록 해주었다. 프랑스는 영감을 주었고 미국은 공감과 성과를 주었다.10)” 라고 술회했듯이 이혼 후 그녀는 예술가와 사진작가, 시인, 영화 관계자들과 교류함으로써 미국의 현대 예술과 진보된 기술을 접하게 되었으며 이러한 경험은 좀 더 창의적이고 앞서가는 디자인을 할 수 있는 계기를 마련해주었다.",
          "1920년대와 1930년대는 야수파와 입체파, 미래파와 더불어 기존의 예술적 가치와 형식을 부정하고 인간 내면인 무의식의 세계에서 새로운 창조성을 찾아낸 초현실주의가 거의 모든 조형예술에 커다란 영향력을 미치고 있었던 시기였다. 더불어 패션계 역시 모더니즘(modernism)이라는 시대적 흐름의 영향을 받은 많은 디자이너들에 의해 펼쳐진 패션의 현대화가 시작되고 있었는데, 스키아빠렐리는 살바도르 달리와 장 꼭도와 같은 초현실주의 예술가들과 교류하면서 초현실주의적 회화의 정신과 표현형식을 패션에 적극적으로 도입함으로써 예술적 가치와 여러가지 상징성을 지닌 표현성이 짙은 새로운 의미의 의복을 창조하였다.",
          "이렇듯 스키아빠렐리는 어린시절의 아름다웠던 추억들과 인생을 통한 경험, 무한한 호기심과 상상력, 예술에 대한 폭넓은 관심과 이해를 바탕으로 평생 철학과 역사를 공부하였으며 아이디어를 구상화시킴에 있어 과감한 시도를 계속한 결과 그 당시까지의 고정관념으로는 상상할 수 없었던 여러 가지 요소들을 복식에 도입하여 충격을 안겨준 패션의 창작자가 되었다."
        ]
      },
      {
        "section": "Ⅲ. 엘자 스키아빠렐리의 의상에 나타난 장식요소의 표현형식"
      },
      {
        "section": "1. 회화적 장식",
        "text": [
          "본 연구에서 회화적 장식이란 의복을 이루는 기본 요소인 소재와 그것을 통해 나타나는 실루엣을 제외한 디테일의 요소 중에서 스키아빠렐리의 디자인에서 두드러지는 자수, 비딩, 패치워크 기법처럼 평면적이고 회화적인 방식으로 의상의 표면을 장식하는 방법을 지칭하는 용어로 한정하였다.",
          "스키아빠렐리는 완벽하게 재단된 옷은 유행이 지난 후에도 스마트하다고 주장하였으며, 1931년 10월 5일 American Vogue가 \"의상의 목수인 스키아빠렐리는 어깨를 세우고, 어깨 아래서부터 허리, 엉덩이에 이르기까지 뚜렷한 선을 구성했다.\"하고 평하였듯이 단순한 선을 이용한 모던한 감각의 명쾌한 실루엣은 건축적이라 할 만큼 엄격한 선을 유지하였다. 그러나 반면에 자수와 비딩, 패치워크와 같은 디테일을 통한 장식은 매우 섬세하고 정교한 꾸뛰르적인 감각과 함께 초현실주의적 성향을 보여준다."
        ]
      },
      {
        "section": "1) 자수(embroidery)",
        "text": [
          "1979년 9월 보그(Vogue)가 '스키아빠렐리의 영향력'이라는 제목 하에 필라델피아 박물관에 소장되어 있는 의상들을 소개하면서 유럽과 미국에서 그녀의 영향력으로 자수 패션이 유행하고 있음을 시사11)하기도 한 사실에서 알 수 있듯이 스키아빠렐리의 의상에서 자수는 빼놓을 수 없는 중요한 요소이다. 그녀의 의상에서 자수는 화가가 붓으로 그림을 그리는 것과 같은 역할로 소재의 문양을 제외한 대부분의 평면적 장식은 자수를 통해 이루어졌다. <그림 1>은 18세기의 아비 아 라 프랑세스(habit à la française)의 디자인을 응용한 짙은 자주색 벨벳의 이브닝 재킷으로 풍성하고 화려한 자수 장식이 돋보인다."
        ]
      },
      {
        "section": "2) 비딩(beading)",
        "text": [
          "스키아빠렐리의 의상에서 비딩은 자수와 함께 사용되어 우아하고 섬세한 표현으로 꾸뛰르적인 아름다움을 보여주는 경우가 많았는데 <그림 2>의 검정 벨벳 케이프의 모티브는 베르사이유 궁전에 있는 분수대에서 영감을 받은 사륜마차와 말로 황금색 시퀸과 원통형 모양의 구슬(bugle), 비즈와 금실, 은실로 수놓아졌다. 이러한 자수 모티브는 상체를 강조하는 스키아빠렐리 디자인의 특성을 강조함과 동시에 다른 의상에 비해 상대적으로 공간이 넓은 케이프의 특성을 효과적으로 이용하고 있다고 보여진다."
        ]
      },
      {
        "section": "3) 패치워크(patch-work)",
        "text": [
          "스키아빠렐리 의복의 장식요소 중 패치워크를 이용한 장식은 초현실주의에서의 꼴라쥬를 연상시키는 기법으로 의상의 재질이나 색상과는 다른 소재와 색상을 덧붙임으로써 회화적 장식의 효과를 거두고 있다. <그림 3>의 재킷은 패치워크된 아이템들이 다양하게 등장하였던 1938년의 ‘commedia dell'arte’ 컬렉션에서 발표된 의상으로, 블랙 울에 블랙 벨벳이 패치워크 되어있어 같은 블랙 색상 안에서도 소재의 변화를 주고 있으며 광대의 마스크(harlequin mask) 모양의 블랙 버튼들이 부착되어 조화를 이루고 있다."
        ]
      },
      {
        "section": "2. 입체적 장식",
        "text": [
          "평면적인 느낌으로 의상의 재질과는 뗄 수 없이 하나의 면에 함께 부착되어 있는 형태의 회화적 장식과는 다르게 입체적 장식은 단추처럼 부조나 환조로 이루어진 입체적 부착물이 장식적 역할을 하도록 사용되어 의복의 조형성에 일조하는 경우로 정의하였다.",
          "스키아빠렐리의 장식요소에 있어서 입체적 장식은 단추를 이용한 경우가 대표적으로 단추에 있어서 그녀는 개척자의 역할을 하였다. 그녀의 의상에서 단추는 기능성 뿐 아니라 의복의 조형적 아름다움을 좌우하는 중요한 장식요소이자 창의적 부가가치를 지닌 디자인 요소로 작용하였다. 패션 단추는 주제 면에서 과일, 악기, 물고기, 서커스에 등장하는 동물 등 주변에서 흔히 볼 수 있는 일상적인 것들로부터 초현실적 환상의 구체화까지 매우 다양하였고 소재 또한 알루미늄, 셀룰로이드, 세라믹, 플라스틱 등 기존의 틀을 탈피하여 그 범위를 넓혔다. 더불어 그녀의 단추는 의복과는 동떨어진 이미지를 보여주는 유머러스한 상징의 하나로 자수나 소재의 패턴 모자 등 다른 요소들과도 조화를 이루어 의복의 장식적 효과를 극대화시키고 있다. 이처럼 스키아빠렐리는 가장 단순한 부착물이며 기능적으로 옷을 여미는 부속품에 지나지 않았던 단추를 다양한 주제와 소재의 사용, 전형적 사이즈와 위치·용도의 변경을 통해 의상의 조형성을 표현하는 한 부분으로 끌어올려 무한한 가능성을 제시하였다.",
          "<그림 4>의 나비모양의 단추는 오버 사이즈의 입체적인 형태로 플라스틱으로 만들어졌고 재킷의 한쪽 면을 따라 4개가 배치되어 있는데, 위에서부터 아래로 내려올수록 작아지는 나비의 크기와 두 개의 나비가 앉아 있는 모자와의 매치에서 스키아빠렐리 특유의 위트와 디자인 감각을 엿볼 수 있다. 재킷에 부착된 4개의 나비 중 가장 작은 허리 근처에 있는 나비 단추만이 버튼으로서의 기능을 하고 있으며, 일반적인 것에 비해 매우 큰 크기로 절개된 단추구멍 역시 장식적인 목적으로 디자인된 것이다. <그림 5>의 플라스틱으로 제작되어 반투명한 재질감의 매미 단추는 실용적인 목적과 장식적인 목적을 동시에 만족시켜주고 있다. 나비와 매미는 서양에서는 재생과 여성의 상징으로 인식되고 있는데 이는 유충에서 허물을 벗고 성충이 되는 변태과정을 통해 새로운 형상으로 재탄생되는 성장과정 상의 특징에서 비롯된 것이라고 보여진다."
        ]
      },
      {
        "section": "3. 악세서리를 통한 장식",
        "text": [
          "1930년대 여성 패션에서 악세서리는 필수적인 요소로 커스튬 주얼리(costume jewelry)를 비롯해 모자, 장갑, 핸드백 등을 갖추어서 착용하는 것이 일반적인 패션 경향이었다. 스키아빠렐리의 악세서리는 의복의 조형성을 분석함에 있어서 빼 놓을 수 없는 중요한 부분으로 의상과 마찬가지로 대담성, 관능성, 재치를 지녔으며, 살바도르 달리, 장 꼭도, 알베르토 자코메티(Alberto Ciacometti, 1901~1966)같은 예술가들과 함께 공동작업으로 디자인되기도 하였다12)."
        ]
      },
      {
        "section": "1) 모자",
        "text": [
          "스키아빠렐리의 악세서리 중에서도 가장 대담하고 혁신적인 아이템인 모자는 염색한 깃털이나 모피, 플라스틱 등 다양한 소재를 사용하였으며, doll-sized hat(인형 모자), toy hat(매우 작은 사이즈의 모자), 종달새 모양의 모자, 타조형태의 모자 등에서 알 수 있듯이 그 크기나 형태에 있어서도 자유로운 표현을 구사하였다. 스키아빠렐리는 매번의 컬렉션마다 모자 디자인을 위한 스케치를 따로 하여 다양한 디자인의 모자를 선보임으로써 모자가 의상의 조형적 완성에 있어 중요한 요소임을 부각시켰다. <그림 6>은 새둥지의 형태를 응용한 모자로 모피와 염색한 새의 깃털들이 조화를 이루어 구체적인 형태를 표현하고 있는데, 초현실주의적 사고에서 새는 가벼움과 우아함을 상징한다."
        ]
      },
      {
        "section": "2) 장갑",
        "text": [
          "초현실주의에서 두드러지는 주제인 손의 이미지는 스키아빠렐리에 의해 장갑이라는 악세서리로 시각화되었다. 1935년에 맨 레이는 피카소가 그린 장갑처럼 보이는 손을 촬영했는데 스키아빠렐리는 이러한 아이디어를 빨간 뱀피 가죽의 손톱이 달린 블랙과 화이트 두 컬러의 스웨이드 장갑으로 표현하였으며 <그림 7>, 이중 블랙 장갑은 데스크 수트와 함께 착용되었다. 스키아빠렐리의 장갑들은 여우머리 모양, 손톱과 핏줄모양 무늬, 보석반지가 끼워져 있는 것처럼 수가 놓여진 것, 성냥을 소매에서 떼어 손목에서 불을 붙이는 것 같은 흡연장갑 등 기발한 아이디어와 유머를 지닌 악세서리로 의상디자인의 마무리 역할을 담당하였다.",
          "<그림 7> 피카소가 그린 장갑(1935)과 스키아빠렐리의 장갑(1936), shocking, pp.130-131"
        ]
      },
      {
        "section": "3) 커스튬 주얼리(costume jewelry)",
        "text": [
          "스키아빠렐리는 '진품 보석은 본질적으로 아름다운 물체이지만, 조합하는 기술이 요구되지 않기 때문에 대담하고 호전적인 정크 주얼리(Junk Jewellery)13)같은 악세서리가 진품보석보다 더 예술적이며, 그것을 착용하는 여성의 개성을 나타내 주는 패션연출의 미적 표현방식14), 이라고 하였다. 그녀는 당시 주얼리에 있어서는 새로운 소재인 단순한 비즈나 도자기, 알루미늄, 유리, 수정, 플랙시글라스(plexglass)15) 등을 과감히 도입함으로써 보석과 준보석, 비싸고 싼 것, 전통적인 것과 미래지향적인 것을 결합시켜 새롭고 환상적인 악세서리들을 만들었다. <그림 8>에서 보이는 손 모양의 클립은 도금한 구리에 에나멜과 커팅된 유리가 사용되었는데 손 모양 모티브는 1934년 가을 컬렉션에서는 버클로 사용되었고, 다섯 개의 손 세트는 코트와 케이프의 파스닝으로, 그리고 싱글 핸드는 박스 형태의 핸드백 윗부분을 손 하나가 잠그고 있는 형태의 잠금장치로 사용되는 등 다양한 디자인과 용도로 전개되었다."
        ]
      },
      {
        "section": "4) 향수",
        "text": [
          "1934년 스키아빠렐리의 첫 향수가 시판된 이후 최대의 성공작인 1937년 ‘Shocking’의 용기는 의상 디자인의 특징인 각진 어깨와 잘록한 허리에서 영감을 받은 축소형 마네킨의 형태로 디자인되었으며 광고도 당시로서는 대담했던 에로틱한 스케치로 표현되어 시선을 모았다. ‘Shocking’의 용기는 앞면에 줄자로 목을 둘러서 교차시켰으며 허리부분에서 이니셜인 S가 새겨진 버튼으로 고정되어 있으며, 마개는 작은 파스텔 컬러의 자기로 된 부케가 둘러싸고 있고 용기 전체는 글래스 돔에 넣어져 있다<그림 9>. 이러한 디자인은 초현실주의 대표적 오브제인 여성을 상징하며 성적인 도발성을 내포하고 있어 스키아빠렐리 의복 디자인의 특성과 그 안에 내포된 초현실주의적 성향을 동시에 표현하고 있다.",
          "이렇듯 스키아빠렐리 악세서리는 일상의 다양한 소재와의 결합, 역동적인 색상과 과장된 크기, 사실적이고 입체적인 형태 등으로 의상과의 조화로운 조형성을 획득하였으며, 인간 내면에 있는 이성이 지배하지 않는 무의식과 잠재의식의 세계를 은유와 변형 또는 유머와 위트로 표현함으로써 상징적 의미를 갖는 하나의 예술적 오브제를 감상하는 새로운 즐거움을 주었다. 위의 내용을 정리하면 <표 1>과 같다."
        ]
      },
      {
        "section": "Ⅳ. 스키아빠렐리의 의상 디자인에 나타난 장식요소의 상징체계"
      },
      {
        "section": "1. 환상성",
        "text": [
          "1929년 세계 대공항과 세기말적 데카당스(decadence)16)는 전쟁의 조짐으로 인한 사회적 불안 심리를 더욱 고조시켜 현실의 어두운면을 벗어나고자 하는 욕망이 팽배하였으며, 이러한 상황 속에서 초현실주의17)는 일상적인 상태와는 대조적인 정신의 세계에서 의식에 속박되지 않는 상상력의 회복과 인간정신의 해방에 초점을 두고 모든 형식에서의 역설과 부조리성(不條理性)을 바탕으로 신비와 비밀, 놀라움에 대한 의식을 창조하는 예술적 성향을 구축하였다. 스키아빠렐리는 이러한 초현실주의의 심상에서 영감을 발견하였고 다음과 같은 방법을 통해 환상성을 표현하였다."
        ]
      },
      {
        "section": "1) 사물의 은유와 변형",
        "text": [
          "스키아빠렐리는 '서랍이 달린 미로의 비너스(Venus de Milo of Drawers)’ 라 불리는 달리의 그림을 기초로 하여 마치 서랍이 달린 듯한 의상인 데스크 수트(desk suit)<그림 10>를 디자인 하였다. 달리에게 있어서 여성을 상징했던 서랍이라는 모티브를 스키아빠렐리는 여성 복식에 가구로서의 의복과 서랍으로서의 주머니로 데빼이즈망기법을 응용하여 표현함으로써 진짜와 가짜, 입체와 평면, 현실과 환상이라는 상반된 구조를 동시에 수용하는 다채로운 의미 체계를 보여주었다. 데스크 수트는 네이비 블루 벨로어(navy blue velour)의 매우 실용적인 수트로 서랍의 손잡이를 연상케 하는 블랙 플라스틱으로 된 떨어지는 모양의 단추로 장식되었고 크라운이 높고 브림이 말려진 네이비 펠트 모자와 매치되었다. 수직으로 배치된 주머니들은 단추들이 서랍 같아 보이도록 가장자리에 세심하게 자수를 놓아 장식적 효과를 의도하기도 하였다. 1961년 1월의 Life지는 25회 창립기념일에 \"지난 25년간의 패션 역사상 가장 두드러진 매력은 1936년 Elsa의 네이비 블루 수트와 doll-sized hat 을 쓴 모델들로 패션계는 엘자 스키아빠렐리의 영혼을 부활시키기 위해 노력해야 할 것이다.\" 라고 명시18)하기도 하여 그 역사적 중요성을 부각시키기도 하였다."
        ]
      },
      {
        "section": "2) 자연물의 위치전환",
        "text": [
          "초현실주의는 인간을 자연으로부터 분리된 만물의 영장이라는 개념이 아닌 보잘 것 없는 자연의 일부로서 자연과의 합일(合一)을 주제로 한 흔히 자연과의 결합을 통해서 범우주적 신비감을 얻어내려는 시도19)를 하였으며, 그 결과 자연물과 인체의 결합에 의해 신화적이며 불안한 기괴함을 보여주기도 하였다. 스키아빠렐리도 이러한 초현실주의의 자연관에 영향을 받아 자연물의 형상을 차용하여 독특한 아이디어로 환상적이고 초현실적인 이미지를 주는 디자인들을 자주 선보였다. <그림 11>은 투명한 플라스틱 안에 금속으로 만든 곤충들이 원형으로 배치된 디자인의 목걸이(bug necklace)로 착용자는 마치 목 주변에서 곤충들이 움직이는 것과 같은 느낌을 받았다. 이러한 디자인 역시 곤충을 일상적 질서에서 떼어 내어 전혀 그 사물과는 연관성이 없는 복식에 갖다 놓음으로서 심리적 충격을 주는 자연물의 위치 전환을 통한 초현실주의적 표현의 일종으로 해석할 수 있으며, 자연의 세계에 들어온 듯한 환상적인 분위기를 자아낸다."
        ]
      },
      {
        "section": "3) 일상을 벗어난 환상적 주제",
        "text": [
          "매 컬렉션마다 다른 주제를 설정하여 그 주제에 적합한 소재와 기법을 시도하였던 스키아빠렐리의 특성은 초현실주의자들이 사용했던 장소성, 일반적 통념, 합리적 사고의 틀을 벗어나 불합리한 조합을 시도하는 방법과 유사한 면모를 보이는데 이는 의상을 디자인하고 컬렉션을 개최하는 디자이너로서 그녀의 뛰어난 창의력을 보여주는 예 중 하나이다. 각각의 컬렉션마다 설정되었던 다양한 주제들은 의상과 모델, 무대 연출의 모든 면이 조화를 이루는 독창성과 연극적 특성을 지닌 것이었다. 1935년도의 ‘Stop, Look and Listen’ 컬렉션에서는 사리 드레스와 신문이 프린트된 직물을 새롭게 선보였으며, 1937년도에는 바닷가재 드레스와 깔때기 모양의 모자가 등장하였던 ‘Butterfly’ 컬렉션을 비롯하여 ‘Paris 1937’, ‘Italian Comedy’ 등의 주제로 다양한 작업을 전개하였다. 이어 1938년에는 ‘Circus’, ‘A Pagan’, ‘Zodiac’, ‘Commedia dell'arte’ 와 같은 주제의 컬렉션들로 스키아빠렐리 특유의 환상성을 표현한 의상과 악세서리들을 보여주었으며, 1939년의 ‘Musical’ 컬렉션에서는 음표와 악기를 수놓은 이브닝 드레스와 새장 모양의 모자를 선보이기도 하였다. 각 컬렉션의 개최 년도와 주제, 그 내용은 <표 1>과 같다. 이처럼 스키아빠렐리는 특정한 자연물이나 추상적 주제에 한정되었던 의상 디자인의 주제를 평범한 일상물이나 초현실적인 범위에까지 확장시켰다."
        ]
      },
      {
        "section": "4) 대담한 색채구성",
        "text": [
          "1934년 11월 하버스 바자(Happers Bazaar)가 \"색에 있어서 엘자 스키아빠렐리는 여성 폴쁘와레(Paul Poiret, 1979~1944)다.20)”라고 언급할 만큼 색상에 있어서 자유분방하며 뛰어난 감각을 선보였는데 그녀의 색채구성은 크게 스키아빠렐리의 대명사와도 같게 된 쇼킹 핑크와 슬리핑 블루 컬러의 고안과 단순하고 구조적이었던 의상의 선을 보완하는 대담한 색상대비로 나누어 볼 수 있다.",
          "스키아빠렐리는 어린 시절 마음속 깊이 새겨졌던 베드로 성당 사제들의 의식복장의 바이올렛(Violet) 색과 야수파, 입체파, 미래파, 초현실주의의 밝고 자극적인 색들에 영향을 받아 그녀만의 독특한 색의 세계를 구축하였다. 늘 새롭고 독자적인 것을 추구했던 그녀는 자신만의 색조를 만들기 위해 감각적인 면 뿐 아니라 기술적으로도 노력하였던 결과, 핑크에 마젠타를 추가한 듯한 색상인 쇼킹 핑크를 만들었고 그 색은 즉각적인 성공으로 이어져 스키아빠렐리의 대표적 상징이 되었다. <그림 12>에서 보여지는 드레스는 할리퀸 패치워크(harlequin patchwork)를 이용한 것으로 쇼킹 핑크, 레드, 바이올렛의 다이아몬드 문양의 소매가 스키아빠렐리 특유의 색채감각을 보여주고 있다. 또한 주로 검정색과 흰색에 원색이나 금속성 색상을 대비시키는 대담한 색상배치 방법은 마치 폴 쁘아레가 야수파나 러시아 발레단의 화려한 원색을 복식에 도입한 것과 비슷한 경향으로 볼 수 있으며, 이러한 색상의 사용은 감정이나 상상이 배제된 고전적 취미의 속성과 반대되는 현대적이고 화려한 것이었다. 이처럼 그녀는 당시에는 많이 사용되지 않았던 새로운 색채와 색상대비로서 그녀만의 독특한 색채감각을 보여주었으며 상상력을 불러일으키는 신선하고 환상적인 이미지를 표현하였다."
        ]
      },
      {
        "section": "2. 이중성",
        "text": [
          "스키아빠렐리의 의상에서 나타난 이중적 이미지는 초현실주의 회화에서 보여졌던 달리의 편집광적 방법처럼 치밀하게 짜여진 구상에 의해 현실적 착각을 일으킨 더블 이미지나 다중상(多重像)으로 보이는 요소들을 의복에 표현하는 것이라고 할 수 있다."
        ]
      },
      {
        "section": "1) 상징 요소의 혼합",
        "text": [
          "스키아빠렐리는 일반적으로 매번의 패션쇼를 웨딩드레스로 끝맺었는데 <그림 13>에서 보이는 드레스에는 평범한 화이트 대신에 화이트 망사에 다크 블루 비즈로 장식되어 마치 여성의 머리카락 혹은 신화에 나오는 메두사(Medusa)처럼 보이는 베일(veil)을 디자인하였다. 이는 시각적인 측면에서의 이중 이미지 뿐 아니라 상징적 측면에서도 순결한 어린 소녀와 뱀처럼 음흉한 머리카락을 지닌 악마의 이중 이미지를 나타내었으며 한편으로 여성의 웨딩 드레스에 대한 이러한 이중적 의미는 스키아빠렐리 자신의 결혼에 대한 개인적 사고를 드러 낸 것21)이라고도 해석해 볼 수 있다. 위의 경우에서 알 수 있듯이 상징요소의 혼합은 일반적으로 상반되는 혹은 모순되는 이미지로 인식되는 두 가지의 이미지를 한 의상 안에 표현함으로써 충격을 주어 환상적이고 초현실적인 이미지를 느끼게 한다."
        ]
      },
      {
        "section": "2) 형태의 혼합",
        "text": [
          "장 꼭도와의 합동작업으로 제작되어 1937년에 발표된 <그림 14>의 드레스는 골드 메탈과 붉은색의 실로 수놓아져 있으며 핑크 실크 패치워크로 처리된 장미 모티브로 구성되어 있다. 이 디자인에서 우리는 두 가지의 이미지를 동시에 볼 수 있는데, 하나는 마주보고 있는 , 두 사람의 옆 얼굴이고 다른 하나는 꼭대기에 장미가 가득한 홈이 파인 기둥이다. 이와 같은 기법은 초현실주의자들이 시각적 착시를 위해 이용했던 트롱프 레이유와 유사한 미러 이미지(mirror image)로 의복에서 그냥 지나치기 쉬운 뒷면을 주시하게 함과 동시에 보는 이로 하여금 낯선 시각적 충격에 대한 유희적 감정을 느끼게 한다. 또한 초현실주의 작가들이 즐겨 사용한 눈속임이란 뜻의 트롱프뢰이유(Trompel'oeil)22) 기법을 이용한 의상은 스키아빠렐리가 패션계에 발을 들여놓게 되는 동기를 마련해준 기념비적인 작품이다. 그녀는 1927년 11월의 컬렉션에서 트롱프 레이유 기법으로 제작되어 흰색 리본이 실제 리본처럼 착각을 일으키게 하는 검정색의 핸드메이드 스웨터를 선보였다. <그림 15>, 이 스웨터는 1927년 12월 American Vogue 지가 예술적 걸작품으로 평가23)한 바 있으며, 블랙 앤 화이트(black & white) 의 심플하고 모던한 디자인은 그녀의 작품 중 가장 많은 복제품을 양산하며 크게 유행하였다."
        ]
      },
      {
        "section": "3) 표현형식의 혼합",
        "text": [
          "스키아빠렐리 의상의 이중적 구조는 제작 기법을 통해서도 나타났다. <그림 16>은 달리와의 협동 작업으로 디자인한 이브닝드레스로 찢겨진 동물 살점의 환영이 프린트되어 있다. 라이트 블루와 마젠타, 블랙의 실크 크레이프로 되어있는 드레스는 소재의 일부분을 잘라 내어 안과 밖의 색상을 달리 배치함으로써 의상의 겉과 안이라는 구조의 특성을 이용하여 마치 실제로 동물의 살점이 벗겨진 것과 같은 회화적 효과를 표현한 경우이다. 더불어 이러한 표현은 1차 세계대전의 참상에서 볼 수 있는 인간 파괴행위에서의 도피와 더불어 무의식 세계의 추구에 대한 갈망의 상징이라 볼 수 있다."
        ]
      },
      {
        "section": "3. 관능성",
        "text": [
          "초현실주의 작가들은 신체 부위를 응용한 데 빼이즈망 기법을 자주 사용하였는데, 인체는 원래의 모습이 아닌 해체와 재조립을 거친 추상화된 형태로 각 부분은 자율성을 지니며 성적인 상징을 불러일으키는 심리적 상징으로 통용되는 경우가 많았다 기존의 의복개념을 파괴하고 스스로 디자이너라기보다는 예술가라고 주장하였던 스키아빠렐리는 인체의 실루엣에 치중된 기존 의상을 단순한 변형이나 전개에 국한시키지 않고 에너지, 정열, 욕망의 표현매체로서 아이디어를 개발하여 복식에 응용하였다. 1937년에 발표되었던 스키아빠렐리의 얇고 반투명한 모슬린의 바닷가재 드레스 <그림 17>은 중심부에 큰 사이즈의 바닷가재가 파슬리와 함께 배치되어 있는 디자인이다. 이는 1936년에 발표되었던 달리의 '바닷가재 전화기(Lobster Telephone) <그림 17>를 응용한 것으로 1934년에 있었던 달리와 스키아빠렐리의 공동작업 시에 달리가 그렸던 스케치에서부터 출발한 아이디어24)였다. 비치는 얇은 소재의 드레스의 중심부에 위치한 크게 확대된 바닷가재는 전통적 실루엣의 드레스 내부에 숨겨진 성 기관의 상징으로, 보는 이로 하여금 여인의 나체를 연상케 하여 첫눈에 볼 때 성적인 심리상태를 자아내도록 하였다25). 또한 신체 부위 중 입술은 불변하는 성적인 기호의 표시(sign)로서 초현실주의의 심볼로 사용되었으며 독립된 형태나 신체의 다른 부분에 재배치되어 관능적 에로티시즘을 표현하였다26). 엘자 스키아빠렐리의 립스틱 광고에 등장하는 입술 역시 립스틱이라는 사물과 초현실주의적 에로티시즘을 상징하고 있으며 천진한 아기 천사들과 관능적인 붉은 입술이라는 대조적인 상황은 스키아빠렐리 전형의 상반된 표현의 예를 보여주고 있다<그림 18>."
        ]
      },
      {
        "section": "4. 유희성",
        "text": [
          "유희는 재미와 즐거움에서 파생되며 경쾌한 마음의 상태 혹은 중압감에서 벗어난 정신의 해방감을 경험하게 하는 희극적 특성을 지니고 있다. 특히 패션에서의 유희적 특성27)은 '우스꽝스러운, 엉뚱한, 놀라운, 의외의'라는 감성적 유머 뿐 아니라 지적, 정신적 즐거움을 주는 효과도 포함된다. 1930년대에 형성된 초현실주의 양식과 결합한 스키아빠렐리의 의상에서의 '재미'의 요소는 의상과는 전혀 상관이 없는 요소들을 의복에 적합한 소재와 형태변형의 과정을 통해 새로운 디자인으로 바꿈으로써 꾸뛰르 세계에 유머와 예술의 조우를 보여주는 새로운 역할을 제시하였다."
        ]
      },
      {
        "section": "1) 형태의 왜곡",
        "text": [
          "유희적 복식에서 형태의 왜곡은 인체를 바탕으로 하는 의복이 기존의 사실적인 관념을 벗어나 형태를 의도적으로 변형, 축소, 과장, 반복하거나 위치의 전환과 재배열함으로써 유희적 감정을 수반하는 경우이다. 이것이 특히 잘 나타난 것이 모자 디자인으로 모자는 초현실주의에서 의식적인 사고와 무의식적인 꿈의 영상이 동시에 존재하는 곳28)이라는 상징적 의미를 지닌다. <그림 19>는 상·하 위치의 변환과 용도의 변경을 통한 복식의 유희적 표현을 보여주는 예인 하이 힐 슈즈(high-heeled shoe)형태의 모자로 1933년 살바도르 달리가 여성용 구두를 쓰고 찍었던 사진에서 받은 영감으로부터 시작되었다29). 스키아빠렐리는 이처럼 모자와는 전혀 연관성이 없는 물체인 신발을 뒤집어 머리에 쓸 수 있게 만들어 그 위치와 형태를 변환시킴으로써 엉뚱하고 우스꽝스러운 유희적 감성을 보여주고 있으며, 이는 초현실주의자들이 즐겨 사용했던 데빼이즈망 기법과도 연결된다."
        ]
      },
      {
        "section": "2) 이질적 요소의 도입",
        "text": [
          "이질적 요소의 도입은 초현실주의의 근본원리이기도 하며 20세기 패션에서 기존의 가치와 형식을 붕괴하는 새로운 방법으로 복식의 유희성을 전달하는 표현 기법중 하나가 되었다. 이러한 방식은 전통적으로 사용되어 온 사물이나 소재에 이질적이고 의외적인 사물이나 이미지를 혼합하는 꼴라쥬에서 영향을 받은 것으로, 부조화한 이미지나 대상의 병치, 의외의 기술이나 소재의 도입, 이미지의 불일치로 웃음을 야기시킬 뿐 아니라 직물재료와 의복 범주의 확대, 성에 대한 고정된 인식의 재구성을 보여준다. <그림 20>은 18세기의 웨이스트 코트를 응용하여 디자인한 재킷으로 야채들이 수놓아진 앞부분과 조화를 이루는 당근과 컬리플라워(cauliflower) 화이트 레디쉬(white radish) 형태의 단추들을 볼 수 있는데, 의복과는 상관성이 없는 물체인 야채류의 형태를 의복의 중요한 디테일인 단추에 배치시킴으로써 의외성을 지닌 재미를 유발하고 시선을 집중시키는 효과를 자아낸다.",
          "이상에서 살펴본 스키아빠렐리 의상 디자인에 내제된 상징체계를 정리하면 <표 2>와 같다."
        ]
      },
      {
        "section": "Ⅴ. 결론",
        "text": [
          "엘자 스키아빠렐리는 1920-30년대에 살바도르 달리와 장 꼭도, 맨 레이 등과의 공동작업을 통하여 반이성적이고 비현실적인 환상의 세계를 보여주었던 초현실주의를 의상 디자인에 상징적으로 표현하였다. 그녀는 패션을 예술로 간주하였으며 인간 내면의 자유로운 상태를 표현하였던 유희적 특성을 보여주는 '재미'의 요소를 기발한 아이디어로 의복에 도입함으로써 고전적 복식미의 규범을 탈피, 은유와 유머라는 새로운 미의 개념을 도입하여 패션을 새로운 예술의 한 장르로 확장시키는 계기를 마련하였다. 본 연구는 단순하고 현대적인 감각을 지닌 스키아빠렐리의 의상 디자인과 함께 독특한 미감을 형성하고 있는 다양한 장식적 요소의 표현형식을 분석하고 그 속에 내재되어 있는 초현실주의적 상징체계의 의미분석에 목적을 두고 전개하였으며 이에 도출된 결론은 다음과 같다.",
          "첫째, 예술적 성향이 뛰어난 엘자 스키아빠렐리의 디자인의 배경에는 성장과정에서의 다양한 예술적 접촉과 왕성한 호기심, 뉴욕과 파리에서의 다양한 예술가들과의 교류와 같은 개인적인 경험들이 뒷받침되었음을 알 수 있었다.",
          "둘째, 스키아빠렐리의 의상에 나타난 장식적 요소는 자수와 비딩, 패치워크 기법으로 표현된 회화적 장식과 환상적이고 기발한 아이디어로 예술적 가치를 지니는 단추를 이용한 입체적 장식 그리고 모자와 장갑, 커스튬 주얼리, 향수 등의 다양한 악세서리로 보여지는 장식으로 나눌 수 있었는데, 이러한 것들은 일상적 요소가 주는 위트와 유머를 제공하는 하나의 오브제로서 감상의 즐거움을 주었으며 기능적 역할이 지배적이었던 기존의 영역에 효과적인 장식과 예술적 가치라는 새로운 의미를 부여하였다.",
          "셋째, 스키아빠렐리의 의상에 나타난 장식요소의 상징체계 중 환상성은 역설과 부조리성을 바탕으로 신비와 비밀, 놀라움에 대한 의식을 창조하였던 초현실주의 예술의 영향으로 나타났으며, 스키아빠렐리의 의상에서는 은유와 변형, 위치의 변경을 이용한 기법으로 표현되었다. 이중성은 더블 이미지나 다중상으로 보이는 요소들을 의복에 표현하는 것으로 상징요소의 혼합 , 형태의 혼합, 표현형식의 혼합으로 분석되어졌다. 또한 관능성은 초현실주의 예술에서 성적 상징을 지녔던 신체부위의 응용을 통해 나타나고 있었는데, 스키아빠렐리는 이를 통해 현대 의상을 인체의 실루엣에 치중된 기존의 형태에서 벗어나 에너지, 정열, 욕망의 표현매체로서 변형·개발하였음을 알 수 있다. 마지막으로 유희성은 기존의 사실적인 관념을 벗어나 형태를 의도적으로 변형, 축소, 과장, 반복하거나 꼴라쥬에서 영향을 받은 부조화한 이미지나 대상의 병치, 의외의 기술이나 소재의 도입을 통해 나타났고 이는 직물재료와 의복 범주의 확대에 영향을 미쳤음을 확인할 수 있었다.",
          "본 연구에서 제시한 스키아빠렐리 의상에 나타난 장식요소의 다양한 표현들은 최근 패션계의 지배적 동향인 의복 장식의 맥시멈(maximum) 현상과 초현실주의적 이미지의 다양한 차용과도 연계되며 현대의상이 외형적 아름다움과 더불어 갖추어야 할 철학적 깊이에 대한 연구에도 참고 자료가 될 것이라 생각된다."
        ]
      }
    ],
    "ref_entries": {
      "FIGURE[1]": {
        "text": "<그림 1> 화려한 자수 장식의 자켓, shocking p.167",
        "type": "figure"
      },
      "FIGURE[2]": {
        "text": "<그림 2> 골드 시퀸으로 비딩된 케이프, 1937, shocking p.191",
        "type": "figure"
      },
      "FIGURE[3]": {
        "text": "<그림 3> 아플리케 장식을 이용한 재킷, 1939, shocking, p.198",
        "type": "figure"
      },
      "FIGURE[4]": {
        "text": "<그림 4> 플라스틱 나비 단추, 1937, shocking, p.150",
        "type": "figure"
      },
      "FIGURE[5]": {
        "text": "<그림 5> 플라스틱 매미 단추, 1938, couture sewing techniques, p.150",
        "type": "figure"
      },
      "FIGURE[6]": {
        "text": "<그림 6> 새둥지 형태를 응용한 모자, 1947, shocking, p.270",
        "type": "figure"
      },
      "FIGURE[7]": {
        "text": "<그림 8> 손 모양의 클립(4.5×2 ), 1936, shocking, p.128",
        "type": "figure"
      },
      "FIGURE[8]": {
        "text": "<그림 9> 향수 shocking, http://www.schiaparelli.com",
        "type": "figure"
      },
      "TABLE[1]": {
        "text": "<표 1> 스키아빠렐리의 의상에 나타난 장식요소의 표현형식",
        "type": "table"
      },
      "FIGURE[9]": {
        "text": "<그림 10> 데스크 수트, 1936, Elsa Schiaparelli p.141",
        "type": "figure"
      },
      "FIGURE[10]": {
        "text": "<그림 11> 곤충이 배치된 원형 목걸이, shocking, p.184",
        "type": "figure"
      },
      "FIGURE[11]": {
        "text": "<그림 12> 쇼킹 핑크, 레드, 바이올렛의 색상 구성으로 된 소매의 드레스, 1939 shocking, p.199",
        "type": "figure"
      },
      "FIGURE[12]": {
        "text": "<그림 13> 이중 이미지를 지닌 웨딩드레스의 베일, 1938, shocking, p.178",
        "type": "figure"
      },
      "FIGURE[13]": {
        "text": "<그림 14> 미러 이미지로 장식된 드레스의 뒷면, 1937, shocking, p.140",
        "type": "figure"
      },
      "FIGURE[14]": {
        "text": "<그림 15> 뜨롱프 레이유 기법을 사용한 스웨터, shocking, pp.18-19",
        "type": "figure"
      },
      "FIGURE[15]": {
        "text": "<그림 16> 찢겨진 동물의 살점이 프린트되어 있는 드레스, 1938, shocking, p.138",
        "type": "figure"
      },
      "FIGURE[16]": {
        "text": "<그림 17> 달리의 바닷가재 전화기와 스키아빠렐리의 바닷가재 드레스, 1936, shocking, p.135",
        "type": "figure"
      },
      "FIGURE[17]": {
        "text": "<그림 18> 엘자 스키아빠렐리의 립스틱 광고, Elsa Schiaparelli p.155",
        "type": "figure"
      },
      "FIGURE[18]": {
        "text": "<그림 19> shoe hat, 19378, shocking, p.137",
        "type": "figure"
      },
      "FIGURE[19]": {
        "text": "<그림 20> 당근, 칼리플라워, 화이트 레디쉬 모양의 단추, 1941, shocking, p.245",
        "type": "figure"
      },
      "TABLE[2]": {
        "text": "<표 2> 스키아빠렐리의 컬렉션별 주제",
        "type": "table"
      },
      "TABLE[3]": {
        "text": "<표 3> 엘자 스키아빠렐리의 의상에 나타난 장식요소의 상징체계",
        "type": "table"
      }
    }
  }


In [84]:
for key in papers_dataset:
  print(key)

doc_id
title
authors
journal
year
abstract
keywords
body_text
ref_entries


# Data Preprocessing

In [85]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

# 데이터셋에서 모든 텍스트 정보를 포함하는 데이터셋 생성 및 문장 단위 분할 함수
def create_sentence_dataset(papers_dataset):
    sentence_dataset = []

    for section in papers_dataset['body_text']:
        if 'text' in section:
            section_text = " ".join(section['text'])
            # 문장 단위로 분할하고 결과 리스트에 추가
            sentences = sent_tokenize(section_text)
            sentence_dataset.extend(sentences)

    return sentence_dataset

# 문장 데이터셋 생성
sentence_dataset = create_sentence_dataset(papers_dataset)

# 결과 출력 (처음 5개 문장)
for sentence in sentence_dataset[:5]:
    print(sentence)

현대의상은 인간의 내적 미의식을 탐구하는 철학적 접근과 더불어 발전하는 소재와 기술, 다양한 창의적 표현을 통해 조형적 아름다움을 추구하고 있으며, 다른 예술 장르와 결합함으로써 그 범위를 점차 넓혀가고 있다.
엘자 스키아빠렐리(Elsa Schiaparelli, 1890~1973)는 대담한 독창성과 혁신적인 창조력, 초현실주의 철학에 대한 이해로 아티스트 꾸띠뤼에(artist-couturier)의 전형이 되었다1).
그녀는 살바도르 달리(Salvador Dali, 1904~1989), 장 꼭도(Jean Cocteau, 1886~1963), 포토그래퍼 세실 비통(Cecil Beaton, 1904~1980)과 같은 예술가들과 교류하면서 초현실주의적 의미와 기법을 의상에 표현한 디자인들로 1930년대 패션의 변화를 주도하였고2) 유머로서의 패션을 제시하였으며 현대의상을 시대의 예술과 그 정신을 표현하는 창의적 산물이라는 새로운 개념으로 확장시키는데 지대한 업적을 남겼다.
더불어 예술과의 조우를 통해 익혀진 조형감각과 숙련된 기술, 다양한 사물과 소재의 활용을 통해 장식적 측면의 중요성을 인식시키고 그 깊이를 더하는데 이바지하였다.
스키아빠렐리에 대한 선행연구들은 '현대패션에 나타난 트롱쁘-뢰이유(Trompe-loeil, 눈속임)에 관한 연구3)’, ‘Schiaparelli의 의상에 나타난 초현실적 표현에 관한 연구4)’에서 보여진 바와 같이 주로 초현실주의 회화 기법과 관련된 작품의 특성에 중점을 둔 방향으로 진행되어져 왔다.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [86]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import faiss

model = SentenceTransformer('/content/drive/MyDrive/YAI/STSmodel.h5')

# 데이터셋 구조 변경 및 문장 단위 벡터화
sentence_embeddings = model.encode(sentence_dataset, convert_to_tensor=True)

# Faiss 인덱스 생성 및 벡터 추가
dimension = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(sentence_embeddings.cpu()))

In [87]:
input_text = '현대 의상은 소재와 기술의 발전, 다양한 창의적 표현과 함께 인간의 내적 미의식을 탐구하는 철학적 접근을 통해 조형적 아름다움을 추구합니다. 이것은 내가 할 수 있는 일입니다.'

input_sentences = sent_tokenize(input_text)

for input_sentence in input_sentences:
    print(f"입력 문장: {input_sentence}")
    input_sentence_vector = model.encode([input_sentence], convert_to_tensor=True)
    D, I = index.search(np.array(input_sentence_vector.cpu()), 1)  # 상위 1개의 가장 유사한 문장을 찾음

    for i, idx in enumerate(I[0]):
        similar_sentence = sentence_dataset[idx]  # `sentence_dataset`에서 유사한 문장 찾기
        distance = D[0][i]  # 해당 문장과의 거리
        print(f"유사한 문장: {similar_sentence}\n거리: {distance}\n")

입력 문장: 현대 의상은 소재와 기술의 발전, 다양한 창의적 표현과 함께 인간의 내적 미의식을 탐구하는 철학적 접근을 통해 조형적 아름다움을 추구합니다.
유사한 문장: 현대의상은 인간의 내적 미의식을 탐구하는 철학적 접근과 더불어 발전하는 소재와 기술, 다양한 창의적 표현을 통해 조형적 아름다움을 추구하고 있으며, 다른 예술 장르와 결합함으로써 그 범위를 점차 넓혀가고 있다.
거리: 25.289358139038086

입력 문장: 이것은 내가 할 수 있는 일입니다.
유사한 문장: 늘 새롭고 독자적인 것을 추구했던 그녀는 자신만의 색조를 만들기 위해 감각적인 면 뿐 아니라 기술적으로도 노력하였던 결과, 핑크에 마젠타를 추가한 듯한 색상인 쇼킹 핑크를 만들었고 그 색은 즉각적인 성공으로 이어져 스키아빠렐리의 대표적 상징이 되었다.
거리: 169.8348846435547



In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sentence_dataset)

input_sentences = sent_tokenize(input_text)

for input_sentence in input_sentences:
    print(f"입력 문장: {input_sentence}")
    input_sentence_vector = model.encode([input_sentence], convert_to_tensor=True)

    # 유클리디언 거리 기반으로 가장 유사한 문장 찾기
    D, I = index.search(np.array(input_sentence_vector.cpu()), 1)

    # 코사인 유사도 계산
    cosine_similarities = util.pytorch_cos_sim(input_sentence_vector, sentence_embeddings)

    # TF-IDF 유사도 계산
    input_tfidf_vector = tfidf_vectorizer.transform([input_sentence])
    tfidf_cosine_similarities = cosine_similarity(input_tfidf_vector, tfidf_matrix).flatten()

    for i, idx in enumerate(I[0]):
        similar_sentence = sentence_dataset[idx]
        distance = D[0][i]
        cosine_similarity_score = cosine_similarities[0][idx].item()
        tfidf_similarity_score = tfidf_cosine_similarities[idx]

        print(f"유사한 문장: {similar_sentence}\n유클리디언 거리: {distance}\n코사인 유사도: {cosine_similarity_score}\nTF-IDF 유사도: {tfidf_similarity_score}\n")

입력 문장: 현대 의상은 소재와 기술의 발전, 다양한 창의적 표현과 함께 인간의 내적 미의식을 탐구하는 철학적 접근을 통해 조형적 아름다움을 추구합니다.
유사한 문장: 현대의상은 인간의 내적 미의식을 탐구하는 철학적 접근과 더불어 발전하는 소재와 기술, 다양한 창의적 표현을 통해 조형적 아름다움을 추구하고 있으며, 다른 예술 장르와 결합함으로써 그 범위를 점차 넓혀가고 있다.
유클리디언 거리: 25.289358139038086
코사인 유사도: 0.8686414361000061
TF-IDF 유사도: 0.5560337120598015

입력 문장: 이것은 내가 할 수 있는 일입니다.
유사한 문장: 늘 새롭고 독자적인 것을 추구했던 그녀는 자신만의 색조를 만들기 위해 감각적인 면 뿐 아니라 기술적으로도 노력하였던 결과, 핑크에 마젠타를 추가한 듯한 색상인 쇼킹 핑크를 만들었고 그 색은 즉각적인 성공으로 이어져 스키아빠렐리의 대표적 상징이 되었다.
유클리디언 거리: 169.8348846435547
코사인 유사도: 0.13297489285469055
TF-IDF 유사도: 0.0

